In [ ]:
import numpy as np
import pandas as pd
import nd2reader
import matplotlib.pyplot as plt
import holoviews as hv
import skimage.filters
import skimage.feature
import scipy.ndimage
import peakutils
from tqdm import tnrange, tqdm_notebook
import dask
import dask.array as da
import distributed
from distributed import Client, LocalCluster, progress
from dask_jobqueue import SLURMCluster
from cytoolz import partial, compose
from itertools import repeat
from glob import glob
import cachetools
import numpy_indexed
import pickle
import warnings
from numbers import Integral

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from segmentation import *
from util import *
from matriarch_stub import *

In [ ]:
plt.rcParams["figure.figsize"] = (8, 8)
hv.extension("bokeh")

# Run

In [ ]:
dask.config.config["distributed"]["scheduler"]["allowed-failures"] = 20
# dask.config.config['distributed']['worker']['memory'] = {'target': 0.4,
#                                                         'spill': 0.5,
#                                                         'pause': 0.9,
#                                                         'terminate': 0.95}

In [ ]:
cluster = SLURMCluster(
    queue="short",
    walltime="03:00:00",
    memory="8GB",
    local_directory="/tmp",
    log_directory="/home/jqs1/projects/molecule-counting/log",
    cores=1,
    processes=1,
)
# diagnostics_port=('127.0.0.1', 8787),
# env_extra=['export PYTHONPATH=\"/home/jqs1/projects/matriarch\"'])
client = Client(cluster)

In [ ]:
cluster

In [ ]:
cluster.scale(80)

In [ ]:
0

## Run

In [ ]:
funcs = {"mean": np.mean, "median": np.median}
#'p0.05': partial(np.percentile, q=5),
#'p0.20': partial(np.percentile, q=20),
#'p0.70': partial(np.percentile, q=70),
#'p0.95': partial(np.percentile, q=95)}

In [ ]:
fluorescence_filenames = glob("/home/jqs1/paulsson/fluorescence/*.nd2")
phase_filenames = glob("/home/jqs1/paulsson/phase/*_0001.nd2")
sandwich_filenames = glob("/home/jqs1/paulsson/sandwich/*_0001.nd2")

In [ ]:
# dark_frames = nd2_to_dask('/n/scratch2/jqs1/fidelity/190313/calibration/dark_100ms.nd2', 0, 'Dark')
# dark_frame = dark_frames.mean(axis=0)
# dark_std = dark_frames.std(axis=0)

In [ ]:
# gfp_flat = nd2_to_dask('/n/scratch2/jqs1/fidelity/190313/calibration/190313_flatfield_FITC_cfpslide_100ms_20pct.nd2').median(axis=0)
# mcherry_flat = nd2_to_dask('/n/scratch2/jqs1/fidelity/190313/calibration/190313_flatfield_mcherry_gfpslide_100ms_000.nd2').median(axis=0)
# mcherry_flat_lowpower = nd2_to_dask('/n/scratch2/jqs1/fidelity/190313/calibration/190313_flatfield_mcherry_30ms_0.35pct_laser.nd2').median(axis=0)

In [ ]:
flats = {"FITC": fitc_flat, "MCHERRY": mcherry_flat}

In [ ]:
%%time
data_futures = {}
for photobleaching_filename in fluorescence_filenames[:]:
    data_futures[photobleaching_filename] = process_file(
        client, funcs, photobleaching_filename
    )

for photobleaching_filename in phase_filenames[:]:
    segmentation_filename = photobleaching_filename.replace("_0001.nd2", ".nd2")
    data_futures[segmentation_filename] = process_file(
        client,
        funcs,
        photobleaching_filename,
        segmentation_filename=segmentation_filename,
    )

for initial_filename in sandwich_filenames[:]:
    segmentation_filename = initial_filename.replace("_0001.nd2", ".nd2")
    photobleaching_filename = initial_filename.replace("_0001.nd2", "_0002.nd2")
    final_filename = initial_filename.replace("_0001.nd2", "_0003.nd2")
    data_futures[segmentation_filename] = process_file(
        client,
        funcs,
        photobleaching_filename,
        segmentation_filename=segmentation_filename,
        initial_filename=initial_filename,
        final_filename=final_filename,
    )

## Segmentation practice

In [ ]:
np.concatenate((0,), np.random.permutation(labels.max()) + 1)

In [ ]:
filename = "/home/jqs1/paulsson/sandwich/190313_SB1_in_GFP_bleaching_in_CFP_100ms_100pct_laser_again_again.nd2"
# filename = '/home/jqs1/paulsson/fluorescence/190313_mkate_100ms_50pct_laser.nd2'
frame = get_nd2_frame(filename, 0, 0, 0)
frame = invert(frame)
labels = segment(frame)

In [ ]:
plt.imshow(labels)

In [ ]:
%%output size=150
RevImage(frame) + RevImage(permute_labels(labels)).redim(z="z2")

In [ ]:
img = frame[400:500, 1420:1500]

In [ ]:
plt.imshow(segment(img))

In [ ]:
plt.imshow(img)

In [ ]:
img_k1.max()

In [ ]:
img_k1

In [ ]:
img2 = img.astype(np.float64)
# img2 = skimage.filters.gaussian(img2, 0.1)
img_k1, img_k2 = hessian_eigenvalues(img2)
img_frangi = skimage.filters.frangi(img_k1, sigmas=np.arange(0.1, 1.5, 0.5))
plt.imshow(img_frangi)

In [ ]:
plt.imshow(img_frangi > skimage.filters.threshold_otsu(img_frangi))

In [ ]:
list(data.keys())

In [ ]:
# img = bf[:600,:600]
# img = invert(img)
# #img = skimage.transform.pyramid_expand(img, upscale=2, multichannel=False)
# mask = img > skimage.filters.threshold_otsu(img)
# img_labels = skimage.morphology.label(mask)
# img = normalize_componentwise(img, img_labels)

In [ ]:
def segment2(img):
    img_highpass = img - gaussian_box_approximation(img, 5)
    # img2 = skimage.transform.pyramid_expand(img, upscale=2, multichannel=False)
    img_k1, img_k2 = hessian_eigenvalues(img)
    img_frangi = skimage.filters.frangi(
        img_k1.max() - img_k1, sigmas=np.arange(1, 3, 0.1)
    )
    cell_mask = img_frangi > skimage.filters.threshold_otsu(img_frangi)
    empty_mask = img_k2 < skimage.filters.threshold_otsu(img_k2)
    empty_mask = repeat_apply(skimage.morphology.dilation, 2)(empty_mask)
    mask_cleaned = cell_mask * empty_mask
    mask_cleaned = skimage.morphology.remove_small_objects(mask_cleaned, 3)
    markers = skimage.measure.label(mask_cleaned)
    labels = skimage.morphology.watershed(img_k1, markers, mask=empty_mask)
    return labels

In [ ]:
def segment3(img):
    img_highpass = img - gaussian_box_approximation(img, 5)
    # img2 = skimage.transform.pyramid_expand(img, upscale=2, multichannel=False)
    img_k1, img_k2 = hessian_eigenvalues(img)
    img_frangi = skimage.filters.frangi(
        img_k1.max() - img_k1, sigmas=np.arange(0.1, 1.5, 0.1)
    )
    cell_mask = img_frangi > skimage.filters.threshold_otsu(img_frangi)

    # empty_mask = (img_k2 < skimage.filters.threshold_otsu(img_k2))
    # empty_mask = repeat_apply(skimage.morphology.dilation, 2)(empty_mask)
    # mask_cleaned = cell_mask * empty_mask
    # mask_cleaned = skimage.morphology.remove_small_objects(mask_cleaned, 3)
    # markers = skimage.measure.label(mask_cleaned)
    # labels = skimage.morphology.watershed(img_k1, markers, mask=empty_mask)
    return labels

In [ ]:
frame = get_nd2_frame(
    "/home/jqs1/paulsson/sandwich/190313_SB1_in_GFP_bleaching_in_mCherry_100ms_100pct_laser.nd2",
    0,
    0,
    0,
)

In [ ]:
plt.imshow(skimage.feature.canny(img_frangi, sigma=2))

In [ ]:
peak_local_max?

In [ ]:
img_k2_frangi = skimage.filters.frangi(
    img_k2.max() + img_k2, sigmas=np.arange(1, 5, 0.5)
)
img_k2_frangi = skimage.filters.gaussian(img_k2_frangi, 2)
maxes = skimage.feature.peak_local_max(img_k2_frangi, min_distance=1, indices=False)
plt.imshow(maxes)

In [ ]:
plt.imshow(img)

In [ ]:
img = frame[:300, :300]
img = invert(img)
# img = skimage.transform.pyramid_expand(img, upscale=2, multichannel=False)
img_highpass = img - gaussian_box_approximation(img, 5)
img_k1, img_k2 = hessian_eigenvalues(img_highpass)
# img_k1 = skimage.filters.gaussian(img_k1, 0.5)
# img_frangi = skimage.filters.frangi(img_k1.max()-img_k1, sigmas=np.arange(0.1,1.5,0.5))
img_frangi = skimage.filters.frangi(
    img_k1.max() - img_k1, sigmas=np.arange(0.1, 1.5, 0.1)
)
markers = img_frangi < skimage.filters.threshold_otsu(img_frangi)
# empty_mask = (img_k2 < skimage.filters.threshold_otsu(img_k2))
# empty_mask = repeat_apply(skimage.morphology.dilation, 2)(empty_mask)
# mask_cleaned = cell_mask * empty_mask
# mask_cleaned = skimage.morphology.remove_small_objects(mask_cleaned, 3)
# markers = skimage.measure.label(mask_cleaned)
# labels = skimage.morphology.watershed(img_k1, markers, mask=empty_mask)

# TODO: replace otsu threshold with mean + 3*stddev of background component
plt.figure()
plt.imshow(img_k1)
plt.figure()
plt.imshow(img_frangi)
plt.figure()
plt.imshow(markers)

In [ ]:
frame = get_nd2_frame(
    "/home/jqs1/scratch/fidelity/190311/190311_205a_10ms_laser100pct_000.nd2", 0, 0, 0
)
labels = segment(frame)

In [ ]:
plt.imshow(img[:300, :300])

In [ ]:
plt.imshow(
    skimage.filters.frangi(
        10 - img_k1[:300, :300], scale_range=(0.5, 1.5), scale_step=0.1
    )
)

In [ ]:
def segment_old(img):
    img_frangi = skimage.filters.frangi(img, scale_range=(0.1, 1.5), scale_step=0.1)
    mask = img_frangi < np.percentile(img_frangi, 90)
    mask = skimage.segmentation.clear_border(mask)
    labels = skimage.measure.label(mask)
    return labels

In [ ]:
imgs = {filename: data[filename][0]["segmentation_frame"] for filename in data.keys()}

In [ ]:
# img = imgs['/home/jqs1/paulsson/fluorescence/190313_mkate.nd2']
# img = imgs['/home/jqs1/paulsson/fluorescence/190313_mkate_100ms_50pct_laser.nd2']
# img = get_nd2_frame('/home/jqs1/paulsson/fluorescence/190313_mkate_100ms_100pct_laser.nd2', 0, 0, 0)
# img = get_nd2_frame('/home/jqs1/scratch/fidelity/190311/190311_mGFPmut2_100ms_laser100pct_003.nd2', 0, 0, 0)
# img = get_nd2_frame('/n/scratch2/jqs1/fidelity/190301/jqs_photobleach_100ms_de32_mkate2_0001.nd2', 0, 0, 0)
img = get_nd2_frame(
    "/home/jqs1/scratch/fidelity/190311/190311_205a_10ms_laser100pct_000.nd2", 0, 0, 0
)

In [ ]:
plt.imshow(ks[0] - img_k1)

In [ ]:
img2 = img.astype(np.float64).copy()[300:600, 300:600]
img2 = skimage.transform.pyramid_expand(img2, upscale=2, multichannel=False)

# img_k1, img_k2 = hessian_eigenvalues(img2)
img

# img_frangi = skimage.filters.frangi(img2, scale_range=(0.1,4), scale_step=0.1)
# img_frangi = skimage.filters.sato(img2, sigmas=range(1, 5, 1), black_ridges=True)
# img_frangi = skimage.filters.frangi(img2, sigmas=np.arange(0.1,0.3,0.05), alpha=3, beta=4, gamma=2)
# img_frangi = hessian_eigenvalues(img2)[0]
# mask = img_frangi < np.percentile(img_frangi, 90)
# mask = skimage.segmentation.clear_border(mask)
# mask = repeat_apply(skimage.morphology.erosion, 2)(mask)
# mask = skimage.morphology.remove_small_objects(mask, 5)
# labels = skimage.measure.label(mask)

# img_thresh = img2 > skimage.filters.threshold_otsu(img2)
# distance = scipy.ndimage.distance_transform_edt(img_thresh)
# local_maxes = skimage.feature.peak_local_max(distance,
#                                              footprint=np.ones((3, 3)),
#                                              indices=False)
# markers = skimage.measure.label(local_maxes)
# labels = skimage.morphology.watershed(img_frangi, markers, mask=img_thresh)

In [ ]:
plt.figure()
plt.imshow(img_k1)
plt.figure()
plt.imshow(img_k2)

In [ ]:
img2 = img.astype(np.float64).copy()[:600, :600]
img2 = img2 - gaussian_box_approximation(img2, 5)
# img2 = skimage.transform.pyramid_expand(img2, upscale=2, multichannel=False)
# img2 = skimage.transform.pyramid_expand(img, upscale=2, multichannel=False)
# img2 = img[:300,:300]
img_k1, img_k2 = hessian_eigenvalues(img2)
img_frangi = skimage.filters.frangi(img_k1, scale_range=(1, 3), scale_step=0.1)
mask = img_frangi > skimage.filters.threshold_otsu(img_frangi)
empty_mask = img_k2 < skimage.filters.threshold_otsu(img_k2)
empty_mask = repeat_apply(skimage.morphology.dilation, 2)(empty_mask)
mask_cleaned = mask * empty_mask
# mask_cleaned = skimage.morphology.remove_small_objects(mask_cleaned)
plt.figure()
plt.imshow(empty_mask)
plt.figure()
plt.imshow(img2)
plt.figure()
plt.imshow(img_frangi)
plt.figure()
plt.imshow(mask)
plt.figure()
plt.imshow(mask_cleaned)

In [ ]:
img_frangi = skimage.filters.frangi(100 - img_k1, scale_range=(1, 3), scale_step=0.1)

In [ ]:
plt.imshow(img_frangi)

In [ ]:
plt.imshow(img_k1 > skimage.filters.threshold_otsu(img_k1) / 2)

In [ ]:
plt.imshow(img_k1)

In [ ]:
plt.imshow(img_frangi)

In [ ]:
plt.imshow(img2 > np.percentile(img2, 96))

In [ ]:
plt.imshow(labels)

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(img2, cmap="Blues")
plt.figure(figsize=(10, 10))
plt.imshow(img_frangi, cmap="Blues")

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(img2, cmap="Blues")
plt.figure(figsize=(10, 10))
plt.imshow(img_frangi, cmap="Blues")

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(img2, cmap="Blues")
plt.figure(figsize=(10, 10))
plt.imshow(mask)

In [ ]:
%%output size=200
RevImage(img2) + RevImage(img_frangi).redim(z="z2")

In [ ]:
%%output size=150
%%opts Layout {+framewise}
hv.HoloMap(
    {filename: RevImage(data[filename][0]["labels"]) for filename in data.keys()}
) + hv.HoloMap(
    {
        filename: RevImage(data[filename][0]["segmentation_frame"])
        for filename in data.keys()
    }
)

## Save data

In [ ]:
%%time
data = client.gather(data_futures)

In [ ]:
with open("190313photobleaching.pickle", "wb") as f:
    pickle.dump(data, f)

In [ ]:
# time: 96593s
# 65% map_over_labels, 30% get_nd2_frame

# Plotting

In [ ]:
with open("190311photobleaching.pickle", "rb") as f:
    data = pickle.load(f)

In [ ]:
list(trace_res.keys())

In [ ]:
b = trace_res["/n/scratch2/jqs1/fidelity/190311/190311_205a_10ms_laser10pct_001.nd2"]

In [ ]:
plt.plot(np.log(b[0]["mean"][10]))

In [ ]:
b[1].hist("area", bins=100)

In [ ]:
traces = z["/home/jqs1/paulsson/fluorescence/190313_GFP_100ms_50pct_laser.nd2"][0][
    "traces"
]["mean"]

In [ ]:
traces = b[0]["mean"]

In [ ]:
idxs = np.random.permutation(traces.shape[0])

In [ ]:
%%output size=250
curves = [
    {
        "x": np.arange(traces.shape[1]),
        "y": np.log(traces[i] / traces[i, 0]),
        "i": idxs[i],
    }
    for i in range(traces.shape[0] // 10)
]
hv.Contours(curves, vdims=["i"]).options(color_index="i", cmap="Category20")

In [ ]:
%%output size=250
hv.Path((np.arange(traces.shape[1]), traces.T))

In [ ]:
%%output size=250
hv.Overlay.from_values([hv.Curve(t) for t in traces])

In [ ]:
%%output size=250
hv.Image(img) + hv.Image(segment(img))